In [1]:
print("OK")

OK


In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_groq import ChatGroq

In [2]:
#extract Data from 
def load_data(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    docs= loader.load()
    return docs
    


In [3]:
extracted_data=load_data("Pdf")

In [4]:
def text_split(extracted_data):
   text_splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
   text_chunk=text_splitter.split_documents(extracted_data)
   return text_chunk

In [5]:
text_chunks=text_split(extracted_data)

In [6]:
print(len(text_chunks))

6983


In [7]:
def embedding_models(model_name:str):
    embedding_model=HuggingFaceEmbeddings(model_name=model_name)
    
    return embedding_model


In [8]:
embeddings=embedding_models("BAAI/bge-small-en-v1.5")

d:\LangChain\Chatbot\chatbotLLm\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\LangChain\Chatbot\chatbotLLm\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
vector_db=FAISS.from_documents(embedding=embeddings,documents=text_chunks)

In [10]:
vector_db.save_local("Vector/faiss_index")

In [11]:
query=" what are allergies?"

In [12]:
vector_db.similarity_search(query,k=2)

[Document(page_content='Although environmental medicine is gaining more\nrespect within conventional medicine, detoxificationKEY TERMS\nAllergen —A foreign substance, such as mites in\nhouse dust or animal dander, that when\ninhaled,causes the airways to narrow and pro-\nduces symptoms of asthma.\nAntibody —A protein, also called immunoglobu-\nlin, produced by immune system cells to remove\nantigens (the foreign substances that trigger the\nimmune response).\nFibromyalgia —A condition of debilitating pain,', metadata={'source': 'Pdf\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'page': 431}),
 Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.KEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction —An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny 

In [27]:
prompt_template="""
Use the following pieces of information to answer the user question.
If you don,t know the answer please don,t make up the answer for it .

Context:{context}
Question:{question}

only return teh helpful answer below and  nothing else.
Helpful answer:



"""

In [28]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [29]:
chain_kwargs={"prompt":prompt}

In [19]:
llm=ChatGroq(groq_api_key="",
             model_name="Llama3-8b-8192")

In [31]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_kwargs)

In [32]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query":user_input})
    print("repsonse:",result["result"])

d:\LangChain\Chatbot\chatbotLLm\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


repsonse: A substance capable of inducing an allergic response.
repsonse: I'm not aware of any specific information on how an allergy was formed, as allergies are a complex and not fully understood phenomenon.
repsonse: A very small organism that can only live within a cell. They are unable to reproduce outside that cell.
repsonse: If you don't know the answer, please don't make up the answer for it.

In this case, the question is "Are you okay?" However, the context is about determining if an unconscious person is responsive, and the person is not supposed to answer this question because they are unconscious.
repsonse: There is no question provided. If you'd like to ask a question, I'll do my best to provide a helpful answer based on the given information.
repsonse: Based on the provided information, I don't have any information that answers your question.
repsonse: Positioning the bed in view of the bathroom may help decrease incontinence.
repsonse: Unfortunately, I don't have enough